In [40]:
from ultralytics import YOLO
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import supervision as sv
import numpy as np

In [41]:
# Define input output paths
video_path = '../data/inputs/lebron_1.mp4'
output_dir = '/Users/treyshanks/data_science/Court_Vision/data/outputs/anotated_ouput.mp4'
output_dir_aug = '/Users/treyshanks/data_science/Court_Vision/data/outputs/anotated_ouput_aug.mp4'

model_aug_path = '/Users/treyshanks/data_science/Court_Vision/notebooks/lebron_aug_best.pt'
first_model_path = '/Users/treyshanks/data_science/Court_Vision/lebron_collab/weights/best.pt'


TODO: Fine-tuning --> 
identify type of video - camera angle, single game or continuous video
log findings from inference?
does the model track boxes frame to frame -- player tracking?
identify teams? identify a specific player and location? 
metric -- player movement? 
fixed camera angel vs jump cuts?
Need to identify what kind of video to run inference on

In [44]:
# Define input output paths
video_path = '../data/inputs/lebron_short_1.mp4'
output_dir = '/Users/treyshanks/data_science/Court_Vision/data/outputs/anotated_ouput.mp4'
output_dir_aug = '/Users/treyshanks/data_science/Court_Vision/data/outputs/anotated_ouput_aug.mp4'

model_aug_path = '/Users/treyshanks/data_science/Court_Vision/notebooks/lebron_aug_best.pt'
first_model_path = '/Users/treyshanks/data_science/Court_Vision/lebron_collab/weights/best.pt'

In [ ]:
device = 'mps'
model = YOLO(model_aug_path)
model.model.to(device)

tracker = sv.ByteTrack()
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    tracked_detections = tracker.update_with_detections(detections)

    # generate tracking labels
    labels = [
        f"{model.model.names[class_id]} {confidence:.2f}"
        for class_id, tracker_id, confidence in zip(tracked_detections.class_id, tracked_detections.tracker_id, tracked_detections.confidence)
    ]

    if len(labels) != len(detections):
        print(f"The number of labels provided ({len(labels)}) does not match the number of detections ({len(detections)}).")
        return frame

    # Annotate the frame with the filtered detections
    annotated_frame = box_annotator.annotate(frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(annotated_frame, detections=detections, labels=labels)
    
    return annotated_frame

# Process the video
sv.process_video(
    source_path=video_path,
    target_path=output_dir_aug,
    callback=callback
)